In [2]:
# ------------------------------
# Import Libraries - RNN STEP BY STEP
# ------------------------------
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from tensorflow.keras.preprocessing.text import one_hot
import tensorflow as tf


# ------------------------------
# Function to Create RNN Model
# ------------------------------
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units,
                        activation=activation[0],
                        input_shape=input_shape))
    model.add(Dense(dense_units, activation=activation[1]))
    model.compile(loss='mse', optimizer='adam')
    return model


# ------------------------------
# Create Demo Model
# ------------------------------
demo_model = create_RNN(
    hidden_units=3,
    dense_units=3,
    input_shape=(3, 1),         # 3-time steps, 1 feature
    activation=['linear', 'linear']
)

demo_model.summary()


# ------------------------------
# Extract Weights
# ------------------------------
U = demo_model.get_weights()[0]    # Input → Hidden
W = demo_model.get_weights()[1]    # Hidden → Hidden
b = demo_model.get_weights()[2]    # Hidden bias
V = demo_model.get_weights()[3]    # Hidden → Output
c = demo_model.get_weights()[4]    # Output bias

print("\nWEIGHTS:")
print("U =", U)
print("W =", W)
print("b =", b)
print("V =", V)
print("c =", c)


# ------------------------------
# Manual RNN Calculation
# ------------------------------
x = np.array([1, 2, 3])        # 3 time steps
x_input = x.reshape(1, 3, 1)   # reshape for model

# Prediction from model
y_pred_model = demo_model.predict(x_input)
print("\nPrediction from Keras Model =", y_pred_model)


# Manual RNN Computation
h0 = np.zeros(3)   # hidden units = 3

# Step-by-step RNN calculation
h1 = x[0] * U.flatten() + b
h2 = x[1] * U.flatten() + np.dot(h1, W) + b
h3 = x[2] * U.flatten() + np.dot(h2, W) + b

o3 = np.dot(h3, V) + c

print("\nManual Computation:")
print("h1 =", h1)
print("h2 =", h2)
print("h3 =", h3)
print("Output =", o3)


# ------------------------------
# ONE-HOT ENCODING EXAMPLE
# ------------------------------

sent1 = "IT students good"
vocab_size = 50

# Convert words → integer index using hashing trick
int_encoded = one_hot(sent1, vocab_size)
print("\nInteger Encoded Sentence:", int_encoded)

# Reshape correctly: (batch, time_steps, features)
int_encoded = np.array(int_encoded).reshape(1, len(int_encoded), 1)
print("Reshaped for RNN:", int_encoded.shape)


# Simple 1-to-many model
model1_to_m = Sequential()
model1_to_m.add(SimpleRNN(1, activation='linear', input_shape=(len(int_encoded[0]), 1)))
model1_to_m.add(Dense(len(int_encoded[0]), activation='sigmoid'))
model1_to_m.compile(loss='mse', optimizer='adam')

# Prediction
y_pred1 = model1_to_m.predict(int_encoded)
print("\nPrediction for One-Hot Text Input:", y_pred1)


/opt/homebrew/Cellar/jupyterlab/4.4.2_1/libexec/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 3)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27 (108.00 B)

 Trainable params: 27 (108.00 B)

 Non-trainable params: 0 (0.00 B)


WEIGHTS:
U = [[-0.10869241  0.7363583   1.143608  ]]
W = [[ 0.2969867  -0.8874259   0.3525256 ]
 [ 0.8873716   0.39284146  0.24134457]
 [ 0.3526621  -0.24114509 -0.9041452 ]]
b = [0. 0. 0.]
V = [[ 0.48597264  0.3368249   0.34009266]
 [-0.05133367 -0.39500523 -0.36839557]
 [ 0.876215    0.8920615  -0.38248897]]
c = [0. 0. 0.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step

Prediction from Keras Model = [[ 3.274732   2.4385607 -1.125581 ]]

Manual Computation:
h1 = [-0.10869241  0.73635828  1.14360797]
h2 = [0.80706559 1.58266964 1.39262753]
h3 = [1.80915355 1.77877691 2.83816645]
Output = [ 3.27473196  2.43856072 -1.12558104]

Integer Encoded Sentence: [36, 1, 20]
Reshaped for RNN: (1, 3, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step

Prediction for One-Hot Text Input: [[0.75201136 0.84764445 0.72228926]]


In [6]:
# ------------------------------
# IMPORTS - RNN USING EMBEDDINGS
# ------------------------------
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf


# ------------------------------
# LOAD DATA
# ------------------------------
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

print("Original shapes:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)


# ------------------------------
# PAD SEQUENCES
# ------------------------------
max_len = 100   # fixed length

X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test  = pad_sequences(X_test,  maxlen=max_len, padding='post')

print("\nAfter padding:", X_train.shape)


# ------------------------------
# BUILD MODEL (FIXED VERSION)
# ------------------------------

model = Sequential()

# EXPLICIT Input Layer — required in new Keras
model.add(tf.keras.Input(shape=(max_len,)))

# Embedding layer
model.add(Embedding(input_dim=10000, output_dim=32))

# RNN Layer
model.add(SimpleRNN(32, return_sequences=False))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

# Show model summary
model.summary()


# ------------------------------
# COMPILE MODEL
# ------------------------------
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


# ------------------------------
# TRAIN MODEL
# ------------------------------
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=128
)


# ------------------------------
# EVALUATE MODEL
# ------------------------------
loss, acc = model.evaluate(X_test, y_test)
print("\nTest Accuracy:", acc)


Original shapes:
X_train: (25000,)
X_test: (25000,)

After padding: (25000, 100)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 322,113 (1.23 MB)

 Trainable params: 322,113 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5918 - loss: 0.6652 - val_accuracy: 0.7168 - val_loss: 0.5897
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8231 - loss: 0.4094 - val_accuracy: 0.8262 - val_loss: 0.3983
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9140 - loss: 0.2311 - val_accuracy: 0.8354 - val_loss: 0.3967
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9656 - loss: 0.1159 - val_accuracy: 0.8332 - val_loss: 0.4499
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.9862 - loss: 0.0572 - val_accuracy: 0.8278 - val_loss: 0.5452
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8248 - loss: 0.5496

Test Accuracy: 0.8248000144958496
